In [1]:
!pip install -U langchain-openai groq python-dotenv langchain-community pypdf langchain-pinecone -qU  rank_bm25 langchain_classic langchain



[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:

import os
from dotenv import load_dotenv

load_dotenv()

GROQ_API_KEY = os.getenv("GROQ_API_KEY")
GROQ_BASE_URL = os.getenv("GROQ_BASE_URL", "https://api.groq.com/openai/v1")

if not GROQ_API_KEY:
    raise ValueError("Set GROQ_API_KEY in your environment before running this notebook.")

In [3]:
from langchain_openai import ChatOpenAI

_groq_client_kwargs = {
    "api_key": GROQ_API_KEY,
    "base_url": GROQ_BASE_URL,
}

model = ChatOpenAI(
    model="openai/gpt-oss-20b",
    stream_usage=True,
    temperature=0,
    **_groq_client_kwargs,
)

c:\Users\HP\Desktop\rag-assistant\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
messages = [
    (
        "system",
        "You are a helpful translator. Translate the user sentence to French.",
    ),
    ("human", "I love programming."),
]
model.invoke(messages)

AIMessage(content="J'aime la programmation.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 60, 'prompt_tokens': 91, 'total_tokens': 151, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 46, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': None, 'queue_time': 0.156814034, 'prompt_time': 0.004397504, 'completion_time': 0.058542028, 'total_time': 0.062939532}, 'model_provider': 'openai', 'model_name': 'openai/gpt-oss-20b', 'system_fingerprint': 'fp_e23fc997ca', 'id': 'chatcmpl-606732ee-62b9-40ee-9c5c-ab95df23c134', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--fbca3806-b2b3-467c-8686-4f92f1f2a974-0', usage_metadata={'input_tokens': 91, 'output_tokens': 60, 'total_tokens': 151, 'input_token_details': {}, 'output_token_details': {'reasoning': 46}})

In [5]:
from langchain_community.document_loaders import PyPDFLoader,TextLoader
def load_documents(file_path):
    doc=[]
    if file_path.endswith(".pdf"):
        loader = PyPDFLoader(file_path)
        doc.extend(loader.load())
    elif file_path.endswith(".txt"):
        loader = TextLoader(file_path)
        doc.extend(loader.load())
    else:
        raise ValueError(f"Unsupported file type: {file_path}")
    return doc

doc=load_documents("C:\\Users\\HP\\Desktop\\iesc103.pdf")
print(len(doc))


12


In [6]:
print(doc[1])
print(doc[1].metadata)



page_content='ATOMS AND MOLECULES 27
• Weigh the flask with its contents
carefully.
• Now tilt and swirl the flask, so that the
solutions X and Y get mixed.
• Weigh again.
• What happens in the reaction flask?
• Do you think that a chemical r
eaction
has taken place?
• Why should we put a cork on the mouth
of the flask?
• Does the mass of the flask and its
contents change?
Law of conservation of mass states that
mass can neither be created nor destroyed in
a chemical reaction.
3.1.2 LAW OF CONSTANT PROPORTIONS
Lavoisier, along with other scientists, noted
that many compounds were composed of two
or more elements and each such compound
had the same elements in the same
proportions, irrespective of where the
compound came from or who prepared it.
In a compound such as water, the ratio of
the mass of hydrogen to the mass of oxygen is
always 1:8, whatever the source of water. Thus,
if 9 g of water is decomposed, 1 g of hydrogen
and 8 g of oxygen are always obtained.
Similarly in ammonia, n

In [7]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(doc)

print(len(all_splits))

34


In [8]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-small-en"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}
hf = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

C:\Users\HP\AppData\Local\Temp\ipykernel_374256\719859098.py:6: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  hf = HuggingFaceBgeEmbeddings(


In [9]:
vec1=hf.embed_query(all_splits[0].page_content)
print(vec1)
texts = [d.page_content for d in all_splits]

embeddings = hf.embed_documents(texts)
print("Number of embeddings:", len(embeddings))
print("Embedding dimension:", len(embeddings[0]))

[-0.05824345350265503, -0.028089674189686775, 0.006125855725258589, 8.941776468418539e-05, -0.029967326670885086, -0.010645505972206593, 0.04138637334108353, 0.01610487699508667, -0.019130941480398178, -0.012159273959696293, -0.014474350027740002, -0.05122330039739609, -0.014677033759653568, 0.018928324803709984, 0.020318087190389633, -0.02421700954437256, -0.02019602805376053, -0.007553625851869583, -0.034821659326553345, 0.030902037397027016, 0.05043011158704758, -0.030332248657941818, -0.023178767412900925, -0.04014768823981285, 0.01234887633472681, 0.03860250860452652, 0.012754983268678188, -0.06030983105301857, -0.00963905081152916, -0.2342124581336975, 0.052616436034440994, -0.019781744107604027, 0.0014835664769634604, -0.029897278174757957, -0.028286611661314964, 0.009501755237579346, 0.021775180473923683, 0.009412484243512154, -0.04134545475244522, -0.004727241117507219, -0.03327125310897827, -0.004660275764763355, 0.03761763125658035, -0.012482318095862865, -0.0265320297330617

In [10]:
import os
from dotenv import load_dotenv
load_dotenv("C:/Users/HP/Desktop/rag-assistant/.env", override=True)
PINECONE_API_KEY=os.getenv("PINECONE_API_KEY")


from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone
from pinecone import ServerlessSpec
pc = Pinecone(api_key=PINECONE_API_KEY)
index_name = "rag-index-dense"

# Only create if not exists
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        vector_type="dense",
        dimension=len(embeddings[0]),
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )
else:
    print(f"Index '{index_name}' already exists. Skipping creation.")






Index 'rag-index-dense' already exists. Skipping creation.


In [11]:
from pinecone import Pinecone, ServerlessSpec

index = pc.Index("rag-index-dense")


records = []

for i, (doc, emb) in enumerate(zip(all_splits, embeddings)):
    records.append({
        "id": str(i),
        "values": emb,   # your dense embedding
        "metadata": {
            "text": doc.page_content,
            "source": doc.metadata.get("source", "unknown"),
            "chunk_id": i
        }
    })

# 4. Upsert in batches
for i in range(0, len(records), 100):
    batch = records[i:i+100]
    index.upsert(
        vectors=batch,
        namespace="rag-namespace"
    )


In [12]:
from langchain_pinecone import PineconeVectorStore
vectorstore = PineconeVectorStore(
    index_name="rag-index-dense",
    embedding=hf,
    namespace="rag-namespace"
)

In [13]:
dense_retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5}
)


In [14]:

from langchain_community.retrievers import BM25Retriever
bm25_retriever = BM25Retriever.from_documents(all_splits)
bm25_retriever.k = 5   # return top 5 keyword matches
print(bm25_retriever)

vectorizer=<rank_bm25.BM25Okapi object at 0x00000113F173DDF0> k=5


In [15]:
from langchain_classic.retrievers import EnsembleRetriever
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, dense_retriever],
    weights=[0.4, 0.6]
)

In [16]:

from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from langchain_classic.retrievers.document_compressors import CrossEncoderReranker


In [17]:
cross_encoder_model = HuggingFaceCrossEncoder(
    model_name="cross-encoder/ms-marco-MiniLM-L-6-v2"
)

reranker = CrossEncoderReranker(
    model=cross_encoder_model,
    top_n=5
)

In [18]:
from langchain_classic.retrievers import ContextualCompressionRetriever

final_retriever = ContextualCompressionRetriever(
    base_compressor=reranker,
    base_retriever=ensemble_retriever
)



In [19]:
len(final_retriever.invoke("summarize the abstract"))
docs = final_retriever.invoke("what problem does the paper solve?")
print(docs)
for d in docs:
    print("-----\n", d.page_content[:300])



[Document(metadata={'producer': 'PDF Printer / www.bullzip.com / FG / Freeware Edition (max 10 users)', 'creator': 'Bullzip PDF Printer (12.2.0.2905)', 'creationdate': '2022-08-22T20:10:12+05:30', 'author': 'admin', 'moddate': '2025-04-04T09:49:17+05:30', 'title': 'D:\\Textbooks\\Rationalised Books\\0964-Science\\1 Source Files\\Chapter-3\\CHAP 3.pmd', 'source': 'C:\\Users\\HP\\Desktop\\iesc103.pdf', 'total_pages': 12, 'page': 1, 'page_label': '2', 'start_index': 783}, page_content='the mass of hydrogen to the mass of oxygen is\nalways 1:8, whatever the source of water. Thus,\nif 9 g of water is decomposed, 1 g of hydrogen\nand 8 g of oxygen are always obtained.\nSimilarly in ammonia, nitrogen and hydrogen\nare always present in the ratio 14:3 by mass,\nwhatever the method or the source from which\nit is obtained.\nThis led to the law of constant proportions\nwhich is also known as the law of definite\nproportions. This law was stated by Proust as\n“In a chemical substance the elements

In [20]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template("""
You are a helpful assistant. Use ONLY the context below to answer the question.
If the answer is not in the context, say "I don't know."

Context:
{context}

Question:
{question}

Answer:
""")


In [21]:
from langchain_core.output_parsers import StrOutputParser

def rag_answer(query):
    # 1. Retrieve top reranked docs
    docs = final_retriever.invoke(query)

    # 2. Join all text for prompt
    context = "\n\n".join([d.page_content for d in docs])

    # 3. Build pipeline
    chain = prompt | model | StrOutputParser()

    # 4. Run final answer
    return chain.invoke({"context": context, "question": query})


In [22]:
from IPython.display import display, Markdown

def pretty_print(text):
    display(Markdown(text.replace("\n", "\n\n")))
pretty_print(rag_answer("summarize the lesson"))
pretty_print(rag_answer("how to cook pasta"))
pretty_print(rag_answer("Who proposed atomic theory?"))
pretty_print(rag_answer("What does the Law of Conservation of Mass state?"))



**Lesson Summary**



- **Atoms and Valency**  

  - Atoms are the smallest particles of matter.  

  - The *valency* (combining power) of an element is its ability to bond, often described as “hands” or “arms.”  

  - Example: An octopus (O) with eight arms can hold a certain number of humans (H) so that the total arms used equals the total arms of all humans. The resulting formula would be \( \mathrm{OH_n} \) (the subscript gives the number of humans).



- **Polyatomic Ions**  

  - A group of atoms carrying a charge is a *polyatomic ion*.  

  - Common examples: \(\mathrm{OH^-}\), \(\mathrm{NH_4^+}\), \(\mathrm{SO_4^{2-}}\).  

  - Their symbols and charges are listed in Table 3.6.



- **Chemical Formulae**  

  - A formula represents the composition of a compound.  

  - Formulas are built by combining element symbols with their valencies (e.g., \(\mathrm{MgCl_2}\), \(\mathrm{CaO}\), \(\mathrm{Cu(NO_3)_2}\), \(\mathrm{AlCl_3}\)).  

  - The mass ratio of constituents can be derived from valency (e.g., calcium oxide: Ca : O = 5 : 2).



- **Key Laws of Chemical Combination**  

  - **Law of Conservation of Mass**: Mass is neither created nor destroyed in a chemical reaction.  

  - **Law of Definite (Constant) Proportions**: A chemical substance always contains its elements in the same mass ratio (e.g., water’s H : O = 1 : 8).  

  - **Dalton’s Atomic Theory**: Explains these laws by proposing that all matter is composed of indivisible atoms that combine in simple whole‑number ratios.



- **Classroom Activities**  

  - Students use placards with element symbols and valencies to cross‑match and form compound formulas.  

  - Exercises involve calculating percentage composition, predicting product masses, and identifying polyatomic ions and simple compounds.

I don't know.

John Dalton.

The Law of Conservation of Mass states that mass can neither be created nor destroyed in a chemical reaction; the total mass of the reactants equals the total mass of the products.